In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [3]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [4]:
df = pd.read_csv('datasets/dataset_merge.csv')
df = df.set_index("session_id")

df

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod,item_purch,purch_date,0,1,2,3,most_seen_cluster,first_item_cluster,last_item_cluster
session_id,,,,,,,,,,,,,,,,,,,
3,2020-12-18 21:19:48.093,2020-12-18 21:25:00.373,9655,9655,2,2,18,312,156,noche,15085,2020-12-18 21:26:47.986,0.0,0.0,0.0,2.0,3,3,3
13,2020-03-13 19:35:27.136,2020-03-13 19:35:27.136,15654,15654,1,4,20,0,0,noche,18626,2020-03-13 19:36:15.507,0.0,0.0,1.0,0.0,2,2,2
18,2020-08-26 19:15:47.232,2020-08-26 19:18:30.833,4026,18316,3,1,25,163,54,noche,24911,2020-08-26 19:20:32.049,0.0,3.0,0.0,0.0,1,1,1
19,2020-11-02 16:30:36.378,2020-11-02 16:48:39.343,19896,8268,17,4,28,1082,63,tarde,12534,2020-11-02 17:16:45.920,0.0,17.0,0.0,0.0,1,1,1
24,2020-02-26 17:22:48.903,2020-02-26 18:24:32.770,2927,18476,9,30,43,3703,411,tarde,13226,2020-02-26 18:27:44.114,3.0,0.0,5.0,1.0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49976,2021-01-03 15:10:27.864,2021-01-03 15:27:03.675,8114,23287,2,3,28,995,497,tarde,24900,2021-01-03 15:28:38.196,0.0,2.0,0.0,0.0,1,1,1
49980,2021-03-31 20:09:31.139,2021-03-31 20:12:58.553,23034,2881,3,4,37,207,69,noche,3057,2021-03-31 20:14:19.965,0.0,2.0,1.0,0.0,1,1,1
49991,2020-08-29 16:59:45.686,2020-08-29 16:59:45.686,7490,7490,1,4,19,0,0,tarde,7590,2020-08-29 17:01:09.711,0.0,0.0,1.0,0.0,2,2,2


### ¿Cuántos tipos de artículos se ven en cada sesión?

Esto nos da una idea valiosa sobre si el clustering nos está dando una información lógica. Si el clustering se ha realizado correctamente, sería normal esperar que en la mayoría de las sesiones sólo se haya visto un tipo (o cluster) de productos. Si el clustering devolviera resultados aleatorios, no habría muchas sesiones en las que sólo se viera un tipo de producto, porque los productos se habrían asignado a los clusters de forma aleatoria, y cada sesión tendría mayoritariamente más de un tipo de producto.

In [5]:
cluster_assign = pd.read_csv('datasets/cluster_assign.csv')
cluster_assign = cluster_assign[['item_id2', 'Cluster_pred']]

In [6]:
number_of_clusters = 4

df_clusts = df[[str(i) for i in range(0,number_of_clusters)]]
df_clusts = df_clusts.replace(0, np.nan)
df_clusts["distinct"] = df_clusts.count(1)
item_counts = df_clusts["distinct"].value_counts()
print(item_counts)

1    7519
2    2820
3     701
4     132
Name: distinct, dtype: int64


En este caso, buscamos sesiones en las que se haya buscado más de un artículo. Como se puede observar, la diferencia de sesiones en las que se haya visto solo un tipo de artículo y dos tipos disminuye. Esto reafirma lo que hemos demostrado en el punto anterior, además de asegurar que las sesiones que tienen un único tipo de producto no son únicamente sesiones donde solo se ha visualizado un producto.

In [7]:
df_clusts = df[[str(i) for i in range(0,number_of_clusters)]]
df_clusts = df_clusts.replace(0, np.nan)
df_clusts["sum"] = df_clusts.sum(axis=1)
df_clusts = df_clusts[df_clusts["sum"] >= 2.0]
df_clusts.head()
df_clusts["distinct"] = df_clusts.count(1) - 1
df_clusts.head()
item_counts = df_clusts["distinct"].value_counts().sort_index()
print(item_counts)
df_clusts.head()

1    4097
2    2820
3     701
4     132
Name: distinct, dtype: int64


,0,1,2,3,sum,distinct
session_id,,,,,,
3,NaN,NaN,NaN,2.0,2.0,1
18,NaN,3.0,NaN,NaN,3.0,1
19,NaN,17.0,NaN,NaN,17.0,1
24,3.0,NaN,5.0,1.0,9.0,3
28,NaN,NaN,4.0,NaN,4.0,1


## Dividir entre Train - Test

In [8]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size=0.2, random_state=40)
len(train_set),len(test_set)

(8937, 2235)

## Preparar los datos para el modelo

In [9]:
cat_vars = ["first_prod", "last_prod", "time_first_prod", "most_common_cat", "most_seen_cluster", "first_item_cluster", "last_item_cluster"]
num_vars = ["prod_count", "time_diff", "time_per_prod", "count_num_unique_cat"] + [str(i) for i in range(0,number_of_clusters)]

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

train_set_num = train_set[num_vars]

num_pipeline = Pipeline([
        ('std_scaler', StandardScaler())
    ])

In [11]:
from sklearn.preprocessing import OneHotEncoder

train_set_cat = train_set[cat_vars]

#Establish values for each category -> otherwise the encoder does not work for the test set
items=cluster_assign.item_id2.to_list()
cats=list(range(1,74))
clusts=list(range(0,number_of_clusters))
times=['día','madrugada','tarde','noche']

cat_pipeline = Pipeline([
        ('one_hot_encoder', OneHotEncoder(categories=[items,items,times,cats,clusts,clusts,clusts], handle_unknown='ignore'))
    ])

In [12]:
from sklearn.compose import ColumnTransformer

num_attribs = train_set_num.columns
cat_attribs = train_set_cat.columns

full_pipeline_train = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_attribs),
        ('cat', cat_pipeline, cat_attribs)
    ])

train_set_prepared = full_pipeline_train.fit_transform(train_set)
train_set_prepared

<8937x47479 sparse matrix of type '<class 'numpy.float64'>'
	with 134055 stored elements in Compressed Sparse Row format>

## Selección del modelo y entrenamiento

Aquí vamos comparar los resultados que obtenemos con 5 modelos distintos:

 - SGD Classifier
 - Decision Tree Classifier
 - K-Nearest Neighbors Classifier
 - SVC Classifier
 - Random Forest Classifier

 Para evaluar su rendimiento y escoger así el más adecuado, tendremos en cuenta el tiempo de entrenamiento del modelo y su Median Reciprocal Rank. 

### SGD Classifier

In [13]:
from sklearn.linear_model import SGDClassifier

#sgd_class = SGDClassifier(loss='modified_huber', random_state=42)
#sgd_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(sgd_class, open('models/sgd_model.sav', 'wb'))

### Decision Tree Classifier

In [14]:
from sklearn.tree import DecisionTreeClassifier

#tree_class = DecisionTreeClassifier(random_state=42)
#tree_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(tree_class, open('models/tree_model.sav', 'wb'))

### K-Nearest Neighbors Classifier

In [15]:
from sklearn import neighbors

#n_neighbors = 3
#knn_class = neighbors.KNeighborsClassifier(n_neighbors)
#knn_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(knn_class, open('models/knn_model.sav', 'wb'))

### SVC Classifier

In [16]:
from sklearn import svm

#SVC_class = svm.SVC(kernel='linear', probability=True)
#SVC_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(SVC_class, open('models/svc_model.sav', 'wb'))

### Random Forest Classifier

In [17]:
from sklearn.ensemble import RandomForestClassifier

forest_class = RandomForestClassifier(n_estimators=20, random_state=42)
forest_class.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

#pickle.dump(forest_class, open('models/random_model.sav', 'wb'))

RandomForestClassifier(n_estimators=20, random_state=42)

## Evaluación del modelo

#### Probamos cada modelo utilizando el método de Mean Reciprocal Rank. Es el que se utiliza en el concurso RecSys Challenge. Esto nos ayudará a elegir el mejor modelo. 

In [18]:
test_set_prepared = full_pipeline_train.fit_transform(test_set)

**Cálculo del MRR**

El rango recíproco medio (Mean Reciprocal Rank) es una medida estadística para evaluar cualquier proceso que produzca una lista de posibles respuestas a una muestra de consultas, ordenadas por probabilidad de corrección. El rango recíproco de una respuesta a una consulta es el inverso multiplicativo del rango de la primera respuesta correcta: 1 para el primer puesto, 1⁄2 para el segundo, 1⁄3 para el tercero y así sucesivamente. El rango recíproco medio es la media de los rangos recíprocos de los resultados de una muestra de consultas

$$

MRR = \frac{1}{| Q |} \displaystyle\sum_{i=1}^{| Q |} \frac{1}{rank_i}

$$

donde $ \displaystyle {\text{rank}}_{i} $ hace referencia a la posición de rango del primer documento relevante para la consulta i-ésima.

In [19]:
def mean_reciprocal_rank(model, set_to_test, rr = 0, cont = 0):

    predictions = model.predict_proba(set_to_test)

    pred_df_test = pd.DataFrame(predictions)
    pred_df_test.columns = model.classes_

    pred_df_test["session_id"] = test_set.index
    pred_df_test = pred_df_test.merge(df[["item_purch"]], how='inner', on='session_id')
    pred_df_test = pred_df_test[["session_id", "item_purch"] + list(pred_df_test.columns[:-2])]

    for index, row in pred_df_test.iterrows():
        item_purch_act = int(row.iloc[1])
        row = row.iloc[2:]
        row_sorted = row.sort_values(ascending=False)
        items = row_sorted.index.to_list()
        if item_purch_act in items:
            rank = int(items.index(item_purch_act)) + 1
            if rank <= 100:
                rr += 1/rank
        cont += 1

    mrr = rr/cont
    print("The mean reciprocal rank for the " + str(model) + " is " + str(mrr))
    return mrr

In [20]:
#SVC_class = pickle.load(open('models/svc_model.sav', 'rb'))
#mean_reciprocal_rank(sgd_class, test_set_prepared)
#mean_reciprocal_rank(knn_class, test_set_prepared)
#mean_reciprocal_rank(SVC_class, test_set_prepared)
#mean_reciprocal_rank(tree_class, test_set_prepared)
mean_reciprocal_rank(forest_class, test_set_prepared)

The mean reciprocal rank for the RandomForestClassifier(n_estimators=20, random_state=42) is 0.03376718154492405


0.03376718154492405

Classifier | Number of Sessions | Mean Reciprocal Rank | Training Time |
| --- | --- | --- | --- |
SGD | 11172 | 0.026005918927187604 | 17s 300s |
KNN | 11172 | 0.016126264214551485 | 200ms |
SVC | 11172 | 0.015471577711113993 | 2min 51s 800ms |
Decision Tree | 11172 | 0.010970834404048573 | 17s 900ms |
Random Forest | 11172 | 0.03393014598889391 | 18s 900ms |

Basándonos en los resultados que hemos obtenido para cada clasificador, podemos ver que los que obtuvieron los mejores resultados fueron el SGD Classifier y el Random Forest Classifier. El SGD Classifier y el Random Forest Classifier, si bien tienen un tiempo de entrenamiento muy parecido, difieren de forma notable en el valor del MRR. Es por eso que finalmente se opta por usar el Random Forest Classifier, al tener el MRR más elevado.

## Mejores parámetros

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = [
    {'n_estimators': [3, 5, 10, 20, 50, 100], 'max_features': [2, 4, 6, 8], 'max_depth':[3, 5, 7, 9]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

grid_search = GridSearchCV(forest_class, param_grid, cv=5,
                           scoring=mean_reciprocal_rank,
                           return_train_score=True)
#grid_search.fit(train_set_prepared, train_set[["item_purch"]].to_numpy().ravel())

In [24]:
print("the best parameters are:")
print(grid_search.best_params_)

the best parameters are:
{'max_depth': 3, 'max_features': 2, 'n_estimators': 3}


## Feature Importance

In [ ]:
clf = Pipeline(steps=[('preprocessor', full_pipeline_train),
                      ('classifier', RandomForestClassifier(n_estimators=20))])

clf.fit(train_set, train_set[["item_purch"]].to_numpy().ravel())

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  Index(['prod_count', 'time_diff', 'time_per_prod', 'count_num_unique_cat', '0',
       '1', '2', '3'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(categories=[[2,
                                                                                              3,
                                                                                              4,
                                                                                              7,
                                 

In [ ]:
onehot_columns = list(clf.named_steps['preprocessor'].named_transformers_['cat'].named_steps['one_hot_encoder'].get_feature_names(input_features=cat_vars))

In [ ]:
numeric_features_list = list(num_vars)
numeric_features_list.extend(onehot_columns)

In [ ]:
import eli5

eli5.explain_weights(clf.named_steps['classifier'], top=50, feature_names=numeric_features_list, feature_filter=lambda x: x != '<BIAS>')

Weight,Feature
0.0262 ± 0.0045,time_diff
0.0258 ± 0.0034,time_per_prod
0.0217 ± 0.0049,prod_count
0.0206 ± 0.0038,count_num_unique_cat
0.0144 ± 0.0034,time_first_prod_tarde
0.0144 ± 0.0043,3
0.0130 ± 0.0034,1
0.0125 ± 0.0036,time_first_prod_día
0.0122 ± 0.0043,time_first_prod_noche
0.0059 ± 0.0023,time_first_prod_madrugada


## Generate TEST Submission

In [ ]:
df_leader = pd.read_csv('online_submission/test_leaderboard_sessions.csv')

In [ ]:
features = pd.read_csv('datasets/item_features.csv')

leader_session_sorted = df_leader.sort_values(by=['date'])

# First date of session
leader_begin_df = leader_session_sorted.groupby('session_id')['date'].first().rename('start_date')

# First product of session
leader_first_prod=leader_session_sorted.groupby('session_id')['item_id'].first().rename('first_prod')

# Number of products seen in the session
leader_prod_count=leader_session_sorted.groupby('session_id')['item_id'].count().rename('prod_count')

# Last date of the session
leader_end_df = leader_session_sorted.groupby('session_id')['date'].last().rename('end_date')

# Last product of the session
leader_last_prod=leader_session_sorted.groupby('session_id')['item_id'].last().rename('last_prod')

# most common category in these session
leader_join_cat = leader_session_sorted.merge(features, on='item_id', how='left').sort_values(by=['feature_category_id'])
leader_common_cat = leader_join_cat.groupby(['session_id'])['feature_category_id'].value_counts().rename('most_common_cat')
leader_common_cat_df = leader_common_cat.to_frame(name = None).reset_index()
leader_common_cat = leader_common_cat_df.groupby(['session_id'])['feature_category_id'].first().rename('most_common_cat')

# counting number of unique categories for session
leader_count_num_unique_cat = leader_common_cat_df.groupby(['session_id'])['feature_category_id'].count().rename('count_num_unique_cat')


# Concat the datasets
leader_times_df = pd.concat([leader_begin_df, leader_end_df, leader_first_prod, leader_last_prod, 
                      leader_prod_count, leader_common_cat, leader_count_num_unique_cat
                     ],axis=1)

# Proper time format
leader_times_df['start_date']=pd.to_datetime(leader_times_df['start_date'])
leader_times_df['end_date']=pd.to_datetime(leader_times_df['end_date'])

# Calculatind the date difference
leader_times_df['time_diff']=(leader_times_df.end_date-leader_times_df.start_date).astype('timedelta64[s]')

# Calculating the time per product
leader_times_df['time_per_prod']=leader_times_df['time_diff']/leader_times_df['prod_count']

# Adding the time of the day
mask=(leader_times_df.start_date.dt.hour>=0) & (leader_times_df.start_date.dt.hour<7)
leader_times_df.loc[mask,'time_first_prod']='madrugada'
mask=(leader_times_df.start_date.dt.hour>=7) & (leader_times_df.start_date.dt.hour<12)
leader_times_df.loc[mask,'time_first_prod']='día'
mask=(leader_times_df.start_date.dt.hour>=12) & (leader_times_df.start_date.dt.hour<18)
leader_times_df.loc[mask,'time_first_prod']='tarde'
mask=(leader_times_df.start_date.dt.hour>=18) & (leader_times_df.start_date.dt.hour<24)
leader_times_df.loc[mask,'time_first_prod']='noche'

In [ ]:
leader_times_df

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod
session_id,,,,,,,,,,
26,2021-06-16 09:53:54.158,2021-06-16 09:53:54.158,19185,19185,1,3,24,0.0,0.000000,día
200,2021-06-25 12:23:40.811,2021-06-25 12:24:50.692,17089,4758,4,30,19,69.0,17.250000,tarde
205,2021-06-11 00:28:07.058,2021-06-11 00:28:07.058,8194,8194,1,3,25,0.0,0.000000,madrugada
495,2021-06-14 22:13:06.741,2021-06-14 22:13:06.741,6853,6853,1,4,20,0.0,0.000000,noche
521,2021-06-19 13:50:03.090,2021-06-19 13:50:03.090,26471,26471,1,3,25,0.0,0.000000,tarde
...,...,...,...,...,...,...,...,...,...,...
4439446,2021-06-11 17:16:12.243,2021-06-11 17:17:11.565,20770,21396,3,4,19,59.0,19.666667,tarde
4439458,2021-06-09 04:52:02.785,2021-06-09 04:54:45.378,11715,26499,2,4,29,162.0,81.000000,madrugada
4439550,2021-06-02 17:42:40.481,2021-06-02 17:42:40.481,19086,19086,1,28,18,0.0,0.000000,tarde


In [ ]:
item_clust = cluster_assign[['item_id2', 'Cluster_pred']].set_index('item_id2')

df_sess_clust_leader=df_leader.merge(item_clust, left_on='item_id', right_index=True)
df_sess_clust_leader

,session_id,item_id,date,Cluster_pred
0,26,19185,2021-06-16 09:53:54.158,1
13240,239993,19185,2021-06-21 11:22:20.096,1
34682,661695,19185,2021-06-27 09:31:21.179,1
36264,699226,19185,2021-06-07 09:58:16.167,1
42810,830528,19185,2021-06-14 15:26:44.282,1
...,...,...,...,...
226231,4382253,7523,2021-06-18 14:05:33.111,3
227011,4397116,6327,2021-06-23 21:59:23.999,3
227515,4408745,10278,2021-06-06 11:23:56.511,2
228684,4428903,27394,2021-06-16 17:03:01.114,1


In [ ]:
df_clust_count_leader = df_sess_clust_leader.groupby(['session_id', 'Cluster_pred'])['session_id'].aggregate('count').unstack().fillna(0)

In [ ]:
df_clust_count_leader["most_seen_cluster"] = df_clust_count_leader.idxmax(axis=1)

In [ ]:
df_leader=pd.concat([leader_times_df, df_clust_count_leader], axis=1)

In [ ]:
df_leader = df_leader.reset_index().merge(item_clust, how='left', left_on='first_prod', right_on=item_clust.index).set_index('session_id')
df_leader = df_leader.rename(columns = {'Cluster_pred':'first_item_cluster'})
df_leader = df_leader.reset_index().merge(item_clust, how='left', left_on='last_prod', right_on=item_clust.index).set_index('session_id')
df_leader = df_leader.rename(columns = {'Cluster_pred':'last_item_cluster'})

In [ ]:
df_leader

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod,0,1,2,3,most_seen_cluster,first_item_cluster,last_item_cluster
session_id,,,,,,,,,,,,,,,,,
26,2021-06-16 09:53:54.158,2021-06-16 09:53:54.158,19185,19185,1,3,24,0.0,0.000000,día,0.0,1.0,0.0,0.0,1,1,1
200,2021-06-25 12:23:40.811,2021-06-25 12:24:50.692,17089,4758,4,30,19,69.0,17.250000,tarde,0.0,0.0,0.0,4.0,3,3,3
205,2021-06-11 00:28:07.058,2021-06-11 00:28:07.058,8194,8194,1,3,25,0.0,0.000000,madrugada,0.0,1.0,0.0,0.0,1,1,1
495,2021-06-14 22:13:06.741,2021-06-14 22:13:06.741,6853,6853,1,4,20,0.0,0.000000,noche,0.0,0.0,0.0,1.0,3,3,3
521,2021-06-19 13:50:03.090,2021-06-19 13:50:03.090,26471,26471,1,3,25,0.0,0.000000,tarde,0.0,1.0,0.0,0.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4439446,2021-06-11 17:16:12.243,2021-06-11 17:17:11.565,20770,21396,3,4,19,59.0,19.666667,tarde,0.0,0.0,0.0,3.0,3,3,3
4439458,2021-06-09 04:52:02.785,2021-06-09 04:54:45.378,11715,26499,2,4,29,162.0,81.000000,madrugada,0.0,1.0,0.0,1.0,1,3,1
4439550,2021-06-02 17:42:40.481,2021-06-02 17:42:40.481,19086,19086,1,28,18,0.0,0.000000,tarde,0.0,0.0,0.0,1.0,3,3,3


In [ ]:
df_leader = df_leader.rename(columns ={0:'0', 1:'1', 2:'2', 3:'3'})

In [ ]:
cat_vars+num_vars

['first_prod',
 'last_prod',
 'time_first_prod',
 'most_common_cat',
 'most_seen_cluster',
 'first_item_cluster',
 'last_item_cluster',
 'prod_count',
 'time_diff',
 'time_per_prod',
 'count_num_unique_cat',
 '0',
 '1',
 '2',
 '3']

In [ ]:
df_leader.columns

Index(['start_date', 'end_date', 'first_prod', 'last_prod', 'prod_count',
       'most_common_cat', 'count_num_unique_cat', 'time_diff', 'time_per_prod',
       'time_first_prod', '0', '1', '2', '3', 'most_seen_cluster',
       'first_item_cluster', 'last_item_cluster'],
      dtype='object')

In [ ]:
leader_set_prepared = full_pipeline_train.fit_transform(df_leader)

In [ ]:
predictions = forest_class.predict_proba(leader_set_prepared)
#print(predictions)

In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.columns = forest_class.classes_
pred_df

,11,18,26,28,32,44,47,54,74,89,...,28101,28103,28112,28114,28117,28122,28124,28133,28135,28139
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
49996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
49997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
49998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0


In [ ]:
candidate_items_df = pd.read_csv("datasets/candidate_items.csv")
columns = candidate_items_df["item_id"].to_list()
real_columns = [b for b in pred_df.columns if b in columns]

In [ ]:
pred_df = pred_df[real_columns]
pred_df["session_id"] = df_leader.index
pred_df = pred_df[["session_id"] + list(pred_df.columns[:-1])]
pred_df

/var/folders/dc/zzpshy3n5qzdd7ywk2xyx44m0000gn/T/ipykernel_2274/3076393300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df["session_id"] = df_leader.index


,session_id,26,54,93,95,107,112,150,160,192,...,27974,27991,28004,28030,28051,28078,28101,28112,28122,28133
0,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,4439446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49996,4439458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49997,4439550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49998,4439653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
f = open('online_submission/final_submission.csv', 'w')
f.write("session_id,item_id,rank" + '\n')

for index, row in pred_df.iterrows():
    #print(index,row)
    session_id_act = int(row.iloc[0])
    row = row.iloc[1:]
    row_sorted = row.sort_values(ascending=False)
    items = row_sorted.index.to_list()
    #print(items[0])
    #break
    for i in range(1,101):
        f.write(str(int(session_id_act)) + ',' + str(items[i-1]) + ',' + str(i) + '\n')